In [1]:
import pandas as pd
import numpy as np
from Simulation_Class_update_2nd import *

In [2]:
sim_try = simulation()

In [3]:
type(sim_try.AED_Iso['geometry'].loc[1])

shapely.geometry.polygon.Polygon

In [4]:
amb_loc = pd.read_parquet("Data/ambulance_locations.parquet.gzip")
Ambulances = amb_loc[amb_loc['province']=='Vlaams-Brabant']
# Only keep rows that contain 30 as all postla code of Leuven start with 30
Ambulances = Ambulances[Ambulances['departure_location'].str.contains('30')]
Ambulances = Ambulances.reset_index(drop = True)
df = pd.DataFrame()
df['Name'] = Ambulances['departure_location_number']
df['longitude'] = Ambulances['longitude']
df['latitude'] = Ambulances['latitude']

In [5]:
pit_loc = pd.read_parquet("Data/pit_locations.parquet.gzip")
Pit = pit_loc[pit_loc['province']=='Vlaams Brabant/Brabant flamand']
Pit = Pit.reset_index(drop = True)
Pit = Pit[Pit['ambucode']==288]
Pit = Pit.reset_index(drop = True)
df_2 = pd.DataFrame()
df_2['Name'] = Pit['unit_id']
df_2['longitude'] = 4.6716518
df_2['latitude'] = 50.8791702

In [6]:
mug_loc = pd.read_parquet("Data/mug_locations.parquet.gzip")
Mug = mug_loc[mug_loc['province']=='Vlaams-Brabant']
Mug = Mug[Mug['postal_code']==3000]
Mug = Mug.reset_index(drop = True)

# Googled the latitude and longitude
df_3 = pd.DataFrame()
df_3['Name'] = Mug['hospital_id']
df_3['longitude'] = 4.6690603
df_3['latitude'] = 50.8784361

In [7]:
df_vectors = pd.concat([df, df_2, df_3], ignore_index = True)
df_vectors['coordinates'] =  list(zip(df_vectors.loc[:,'longitude'], df_vectors.loc[:,'latitude']))
Vectors = df_vectors[['longitude', 'latitude','coordinates']]

In [8]:
Vectors.head()

,longitude,latitude,coordinates
0,4.724344,50.869655,"(4.7243436, 50.8696549)"
1,4.695398,50.878535,"(4.6953981, 50.8785355)"
2,4.098320,50.868004,"(4.0983199, 50.8680038)"
3,4.472760,50.879821,"(4.4727595, 50.8798215)"
4,4.665862,50.877460,"(4.6658621, 50.8774603)"


In [9]:
intervention = pd.read_csv("Data/interventions.csv")
intervention.rename(columns = {'Longitude intervention':'longitude', 'Latitude intervention':'latitude'}, inplace = True)
intervention['coordinates'] = list(zip(intervention['longitude'], intervention['latitude']))

In [10]:
intervention.head()

,T1,T3,latitude,longitude,Response Time,coordinates
0,2022-06-07 07:02:17,2022-06-07 07:11:24,50.861866,4.708210,547.0,"(4.70821, 50.861866)"
1,2022-06-08 13:26:34,2022-06-08 13:35:03,50.873174,4.716699,509.0,"(4.716699, 50.873174)"
2,2022-06-08 14:35:19,2022-06-08 14:47:58,50.928140,4.703247,759.0,"(4.703247, 50.92814)"
3,2022-06-09 09:51:23,2022-06-09 10:03:05,50.874550,4.730246,702.0,"(4.730246, 50.87455)"
4,2022-06-13 12:35:22,2022-06-13 12:44:12,50.879098,4.696672,530.0,"(4.696672, 50.879098)"


In [11]:
(intervention.loc[0])

T1                2022-06-07 07:02:17
T3                2022-06-07 07:11:24
latitude                    50.861866
longitude                     4.70821
Response Time                   547.0
coordinates      (4.70821, 50.861866)
Name: 0, dtype: object

In [12]:
# Create mock data for responders
center_lat = 50.86186
center_lon = 4.70821

offset = 0.001  

points = [
    (center_lon, center_lat + offset),  # North
    (center_lon, center_lat - offset),  # South
    (center_lon + offset, center_lat),  # East
    (center_lon - offset, center_lat)   # West
]

# Create a DataFrame
df = pd.DataFrame(points, columns=['longitude', 'latitude'])
print(df)

   longitude  latitude
0    4.70821  50.86286
1    4.70821  50.86086
2    4.70921  50.86186
3    4.70721  50.86186


In [13]:
# create a data frame based on the coordinates of the close responders
Responder_df = pd.DataFrame(df, columns =['longitude', 'latitude'])
# also get the coordinates as a tuple
Responder_df['Responder_loc'] = list(zip(Responder_df['longitude'],Responder_df['latitude']))
Responder_df.head()

,longitude,latitude,Responder_loc
0,4.70821,50.86286,"(4.70821, 50.86286)"
1,4.70821,50.86086,"(4.70821, 50.86086)"
2,4.70921,50.86186,"(4.709210000000001, 50.86186)"
3,4.70721,50.86186,"(4.70721, 50.86186)"


In [14]:
AED_location = pd.read_csv("Data/filtered_AED_loc.csv")

In [15]:
AED_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                139 non-null    float64
 1   type              28 non-null     object 
 2   address           139 non-null    object 
 3   number            127 non-null    float64
 4   postal_code       139 non-null    float64
 5   municipality      139 non-null    object 
 6   province          139 non-null    object 
 7   location          72 non-null     object 
 8   public            139 non-null    object 
 9   available         25 non-null     object 
 10  hours             10 non-null     object 
 11  complete_address  139 non-null    object 
 12  latitude          139 non-null    float64
 13  longitude         139 non-null    float64
dtypes: float64(5), object(9)
memory usage: 15.3+ KB


In [16]:
df_result = sim_try.fastest_time(intervention.loc[0], Responder_df, AED_location, Vectors, decline_rate = 1, N_responder=50, opening_hour=12, Dist_responder = 600, Dist_AED = 600, Dist_Vector = 600,
                     distribution = {(6.0, 22.0): 0.5,(0.0, 24.0): 0.05,(8.0, 20.0): 0.3,(9.0, 18.0):0.15})

Comparing responders vs. vectors
        lon        lat                                           geometry  \
0  4.704471  50.856036  POLYGON ((4.69593 50.85836, 4.69593 50.85800, ...   
1  4.705708  50.855550  POLYGON ((4.69752 50.86062, 4.69695 50.85820, ...   
2  4.712295  50.861705  POLYGON ((4.70170 50.86221, 4.70271 50.86044, ...   

                               AED  opening_hour  closing_hour  Patient_lon  \
0  (4.704471000000001, 50.8560364)           8.0          20.0      4.70821   
1          (4.7057077, 50.8555502)           9.0          18.0      4.70821   
2           (4.712295, 50.8617055)           8.0          20.0      4.70821   

   Patient_lat           Patient_loc  open  
0    50.861866  (4.70821, 50.861866)     1  
1    50.861866  (4.70821, 50.861866)     1  
2    50.861866  (4.70821, 50.861866)     1  
Duration for Vectors found


In [18]:
df_result

,Patient_loc,Responder_loc,duration_Responder,AED_loc,duration_AED,Vector_loc,duration_Vector
0,"(4.70821, 50.861866)",No responder,No responder,No AED,No AED,"(4.6998331, 50.8723536)",339.2


In [20]:
decline_rate = 0.5
list(np.random.choice([0,1], p=[decline_rate, (1-decline_rate)], size = 5))

[0, 1, 0, 0, 0]

In [4]:
# Create mock data for responders
center_lat = 50.8797
center_lon = 4.6701

offset = 0.01  

points = [
    (center_lon, center_lat + offset),  # North
    (center_lon, center_lat - offset),  # South
    (center_lon + offset, center_lat),  # East
    (center_lon - offset, center_lat)   # West
]

# Create a DataFrame
df = pd.DataFrame(points, columns=['longitude', 'latitude'])
print(df)

   longitude  latitude
0     4.6701   50.8897
1     4.6701   50.8697
2     4.6801   50.8797
3     4.6601   50.8797


In [21]:
AEDs = pd.read_csv("Data/filtered_AED_loc.csv")

In [22]:
AEDs.head

<bound method NDFrame.head of           id          type               address  number  postal_code  \
0      318.0           NaN      Interleuvenlaan     58.0       3001.0   
1      323.0           NaN           Interleuven    58.0       3001.0   
2      385.0           NaN  Kolonel Begaultlaan     15.0       3012.0   
3      419.0           NaN     Katjeswilgenlaan      NaN       3010.0   
4      427.0           NaN         Tervuursevest   101.0       3001.0   
..       ...           ...                   ...     ...          ...   
134  15338.0  Fixed Device           Kapeldreef     75.0       3001.0   
135  15340.0  Fixed Device            Kapeldreef    75.0       3001.0   
136  15682.0  Fixed Device           Philipssite     NaN       3001.0   
137  16184.0  Fixed Device      Pellenbergstraat   160.0       3010.0   
138  10448.0  Fixed Device      Interleuvenlaan      NaN       3001.0   

    municipality        province                          location public  \
0         Leuven

In [6]:
# create a data frame based on the coordinates of the close responders
Responder_df = pd.DataFrame(df, columns =['longitude', 'latitude'])
# also get the coordinates as a tuple
Responder_df['Responder_loc'] = list(zip(Responder_df['longitude'],Responder_df['latitude']))
Responder_df.head()

,longitude,latitude,Responder_loc
0,4.6701,50.8897,"(4.6701, 50.8897)"
1,4.6701,50.8697,"(4.6701, 50.8697)"
2,4.6801,50.8797,"(4.6800999999999995, 50.8797)"
3,4.6601,50.8797,"(4.6601, 50.8797)"


In [7]:
def assign_decline_rates(possible_responder, target_decline_prob):
    """
    Adjust individual responder decline rates so that the overall probability of all responders declining matches target.
    
    Parameters:
    possible_responder (list): List of responder IDs or locations.
    target_decline_prob (float): Desired probability that all responders decline.
    
    Returns:
    pd.DataFrame: DataFrame with responders and their individual probability of accepting.
    """
    N = len(possible_responder)
    if N == 0:
        raise ValueError("No responders available.")
    
    # Solve for individual decline rate: decline_rate = target_decline_prob^(1/N)
    decline_rate = target_decline_prob ** (1 / N)
    
    # Generate acceptance probabilities
    acceptance = np.random.choice([0, 1], p=[decline_rate, 1 - decline_rate], size=N)
    
    return pd.DataFrame({'Responder_loc': possible_responder, 'Probability': acceptance})


In [8]:
xd = assign_decline_rates(Responder_df, 0.6)

ValueError: 2

In [203]:
x = route_try.send_responders(patient, Responder_df, AEDs)

        lon        lat                                           geometry  \
0  4.680963  50.883906  POLYGON ((4.66959 50.88601, 4.67000 50.88470, ...   
1  4.668996  50.878281  POLYGON ((4.66245 50.87689, 4.66353 50.87490, ...   
2  4.661188  50.875833  POLYGON ((4.65599 50.87288, 4.66057 50.87125, ...   

                               AED  longitude   latitude  Patient_lon  \
0  (4.680962699999999, 50.8839064)   4.680963  50.883906       4.6701   
1           (4.668996, 50.8782805)   4.668996  50.878281       4.6701   
2          (4.6611883, 50.8758335)   4.661188  50.875833       4.6701   

   Patient_lat        Patient_loc  
0      50.8797  (4.6701, 50.8797)  
1      50.8797  (4.6701, 50.8797)  
2      50.8797  (4.6701, 50.8797)  


In [204]:
x

{'coord_direct': (4.6601, 50.8797),
 'coord_AED': (4.6800999999999995, 50.8797),
 'AED_coordinates': (4.680962699999999, 50.8839064)}

In [292]:
dir = route_try.directions([[4.6800999999999995, 50.8797], [4.680962699999999, 50.8839064],[4.6701, 50.8797]])

In [296]:
dir['coordinates']

[[4.680094, 50.879736],
 [4.67993, 50.879725],
 [4.679923, 50.879746],
 [4.67989, 50.879939],
 [4.680292, 50.879958],
 [4.680492, 50.879987],
 [4.680521, 50.879951],
 [4.680872, 50.879958],
 [4.680977, 50.879998],
 [4.681561, 50.8801],
 [4.681677, 50.880137],
 [4.681717, 50.880179],
 [4.681678, 50.880258],
 [4.681666, 50.880365],
 [4.681843, 50.880861],
 [4.6821, 50.881585],
 [4.682735, 50.882599],
 [4.682744, 50.882614],
 [4.682814, 50.882687],
 [4.682786, 50.882772],
 [4.682696, 50.882874],
 [4.682023, 50.883285],
 [4.681902, 50.883446],
 [4.680839, 50.883707],
 [4.68034, 50.883829],
 [4.679681, 50.883976],
 [4.678864, 50.884157],
 [4.678772, 50.884177],
 [4.678536, 50.884227],
 [4.678355, 50.884266],
 [4.678193, 50.884299],
 [4.678082, 50.884323],
 [4.677862, 50.884106],
 [4.677709, 50.884008],
 [4.677502, 50.883918],
 [4.676806, 50.88377],
 [4.676683, 50.883725],
 [4.676622, 50.883645],
 [4.676267, 50.883662],
 [4.676015, 50.883496],
 [4.675934, 50.883442],
 [4.675411, 50.883426],


In [305]:
Responder_df

,longitude,latitude,Responder_loc
0,4.6701,50.8897,"(4.6701, 50.8897)"
1,4.6701,50.8697,"(4.6701, 50.8697)"
2,4.6801,50.8797,"(4.6800999999999995, 50.8797)"
3,4.6601,50.8797,"(4.6601, 50.8797)"


In [210]:
gt = list(Responder_df['Responder_loc'])

In [211]:
gt

[(4.6701, 50.8897),
 (4.6701, 50.8697),
 (4.6800999999999995, 50.8797),
 (4.6601, 50.8797)]

In [214]:
ft = pd.DataFrame([(4.4701, 50.8897), (4.6701, 50.8697), (4.8800999999999995, 50.8797), (4.9601, 50.8797)])
ft['Responder_loc'] = list(zip(ft[0],ft[1]))

In [215]:
ft

,0,1,Responder_loc
0,4.4701,50.8897,"(4.4701, 50.8897)"
1,4.6701,50.8697,"(4.6701, 50.8697)"
2,4.8801,50.8797,"(4.8801, 50.8797)"
3,4.9601,50.8797,"(4.9601, 50.8797)"


In [216]:
f = list(ft['Responder_loc'])

In [217]:
unique = f + gt

In [220]:
seen = set()
unique = [val for val in unique if val not in seen and (seen.add(val) or True)]

In [221]:
unique

[(4.4701, 50.8897),
 (4.6701, 50.8697),
 (4.8801, 50.8797),
 (4.9601, 50.8797),
 (4.6701, 50.8897),
 (4.6800999999999995, 50.8797),
 (4.6601, 50.8797)]

In [223]:
from numpy import random

In [256]:
decline_rate = 0.5

In [257]:
x = list(random.choice([0,1], p=[decline_rate, (1-decline_rate)], size=10))

In [258]:
x

[1, 0, 1, 1, 0, 0, 0, 0, 1, 0]

In [239]:
df = pd.DataFrame({'Responder_loc' : unique,
                                'prob' : x})

In [241]:
df.head()

,Responder_loc,prob
0,"(4.4701, 50.8897)",1
1,"(4.6701, 50.8697)",1
2,"(4.8801, 50.8797)",1
3,"(4.9601, 50.8797)",1
4,"(4.6701, 50.8897)",1


In [269]:
df_duration_indirect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Responder_loc         25 non-null     object 
 1   Patient_loc           25 non-null     object 
 2   latitude              25 non-null     float64
 3   longitude             25 non-null     float64
 4   AED_coordinates       25 non-null     object 
 5   duration_through_AED  25 non-null     int32  
dtypes: float64(2), int32(1), object(3)
memory usage: 1.2+ KB


In [288]:
a = dict()

In [289]:
a.update(df_duration_direct.iloc[1])

In [290]:
a

{'Responder_loc': (-78.290713, 161.598793),
 'Patient_loc': (54.660974, -112.834779),
 'latitude': 73.48785947399767,
 'longitude': -117.22808555820308,
 'duration_direct': 108}

In [ ]:
import openrouteservice
from openrouteservice import client
Client_ors = openrouteservice.Client(key='5b3ce3597851110001cf6248a8348d6a74544602bd7cbc7936c635d1')

In [ ]:
Responder_df['duration_direct']=[self.directions([i, Patient_cood], profile = 'foot-walking')['duration'] for i, 
                                          Patient_cood in zip(Responder_df['Responder_loc'], Responder_df['Patient_loc'])]

In [317]:
direct_index

24

In [322]:
len(df_duration_direct)

25

In [324]:
df_duration_direct.iloc[24]

Responder_loc      (-12.249897, -75.15751)
Patient_loc         (-33.415923, 3.085449)
latitude                        -35.841904
longitude                      -165.320949
duration_direct                        996
Name: 24, dtype: object

In [314]:
indirect_index = 0

In [ ]:
coord_direct = []
coord_AED = []
AED_coordinates = []

direct_index = 0
indirect_index = 0
total_direct = len(df_duration_direct)
total_indirect = len(df_duration_indirect)

while direct_index < total_direct and indirect_index < total_indirect:
    # Handle the direct selection
    direct_row = df_duration_direct.iloc[direct_index]
    if direct_row['Responder_loc'] in used_locations:
        if direct_index+1 < len(df_duration_direct):
            direct_row = df_duration_direct.iloc[direct_index + 1]
            direct_index += 2
        else:
            pass
    else:
        direct_index += 1

    coord_direct.append(direct_row['Responder_loc'])
    used_locations.add(direct_row['Responder_loc'])

    # Handle the indirect selection
    indirect_row = df_duration_indirect.iloc[indirect_index]
    if indirect_row['Responder_loc'] in used_locations:
        if indirect_index+1 < len(df_duration_direct):
            indirect_row = df_duration_indirect.iloc[indirect_index + 1]
            indirect_index += 2
        else:
            pass
    else:
        indirect_index += 1

    coord_AED.append(indirect_row['Responder_loc'])
    AED_coordinates.append(indirect_row['AED_coordinates'])
    used_locations.add(indirect_row['Responder_loc'])

# Add remaining rows if one DataFrame is exhausted
if direct_index < total_direct:
    remaining_direct = df_duration_direct.iloc[direct_index:]
    coord_direct.extend(remaining_direct['Responder_loc'].tolist())

elif indirect_index < total_indirect:
    remaining_indirect = df_duration_indirect.iloc[indirect_index:]
    coord_AED.extend(remaining_indirect['Responder_loc'].tolist())
    AED_coordinates.extend(remaining_indirect['AED_coordinates'].tolist())

# Convert to DataFrame if needed
#result_df = pd.DataFrame(selected_rows)
#print(result_df.head())




IndexError: single positional indexer is out-of-bounds

In [328]:
pd.DataFrame(
    {'Indirect responder': coord_AED,
     'AED': AED_coordinates
    })

,Indirect responder,AED
0,"(59.172752, -51.568802)","(-11.578878, 82.814154)"
1,"(-86.294791, 169.167547)","(-18.212238, -23.932534)"
2,"(37.234322, 82.44258)","(-86.911002, 94.811192)"
3,"(-61.916645, -123.841973)","(35.912198, -130.192887)"
4,"(-81.859088, -62.881081)","(78.23052, 49.777414)"
5,"(48.828662, -153.343925)","(55.244336, -55.330444)"
6,"(-30.438356, -157.118994)","(69.60247, -86.078296)"
7,"(-76.580884, 175.279297)","(-28.46166, -146.952336)"
8,"(49.004058, -108.462355)","(-73.051742, -67.891209)"
9,"(17.621996, 151.874725)","(-10.376599, -93.676551)"


In [ ]:
        fastest_aed = df_duration_indirect.loc[0]
        fastest_gpr = df_duration_direct.loc[0]
        second_fastest_aed = df_duration_indirect.loc[1]
        second_fastest_gpr = df_duration_direct.loc[1]

        # Fastest responder going for the AED
        # Check if the 2nd fastest direct responder is faster than the fastes direct through AED
        if second_fastest_gpr['duration_direct'] < fastest_aed['duration_through_AED']:
            # if so, z equal to CPR by 2nd fastest responder
            CPR_time = fastest_aed['duration_through_AED'] - second_fastest_gpr['duration_direct']
            surv_A = self.survival_probability(second_fastest_gpr['duration_direct'], CPR_time)
        else:
            # - z equals zero as no one does any CPR
            surv_A = self.survival_probability(fastest_aed['duration_through_AED'], 0)
        # 2nd fastest responder arriving with AED
        # - time until AED arrives minus time CPR arrives is the time without CPR
        surv_B = self.survival_probability(fastest_gpr['duration_direct'], second_fastest_aed['duration_through_AED']-fastest_gpr['duration_direct'])    
            
        # Check if the fastest through AED is the same as the fastest direct 
        if fastest_aed['Responder_loc']==fastest_gpr['Responder_loc']:
            # Find best strategy which is the maximal survival chances
            best_strategy = max(surv_A, surv_B)
            # Send responders
            if best_strategy == surv_A:
                # - Second fastest direct time will be send directly  
                # - Fastest direct and AED responder will be send through the AED
                coord_direct = second_fastest_gpr['Responder_loc']
                coord_AED =  fastest_aed['Responder_loc']
                AED_coordinates = fastest_aed['AED_coordinates']
            # If this is not true:
            # - Fastes direct responder will be send directly
            # - Second fastest through AED responder will be send through the AED
            else:
                coord_direct = fastest_gpr['Responder_loc']
                coord_AED = second_fastest_aed['Responder_loc']
                AED_coordinates = second_fastest_aed['AED_coordinates']
        else:
            # If the fastest direct responder and thorugh AED responder are different:
            # - Take the fastest responders for both
            coord_direct = fastest_gpr['Responder_loc']
            coord_AED = fastest_aed['Responder_loc']
            AED_coordinates = fastest_aed['AED_coordinates']

In [ ]:
# Initialize sets and lists
used_locations = set()
selected_rows = []

In [ ]:
n = 25
Amount of people in  = list(range(0,n+1))